# Projeto de sistema de recomendação de filmes

A estrutura do código e ideias que o motivaram encontra-se no arquivo "structure.ipynb".

O dataset utilizado é o MovieLens, cujo detalhamento pode ser encontrado no link http://files.grouplens.org/datasets/movielens/ml-20m-README.html


## Passo 1) Lendo o dataset


Os arquivos são disponibilizados no formato '.csv'. Assim, faremos a leitura dos arquivos com a biblioteca Pandas. 

Deve, neste ponto, também ser separado um conjunto utilizado para a recomendação e outro para a avaliação do sistema. Com o auxílio da biblioteca Scikit-learn, o método de divisão estratificada será utilizado para manter a proporção de avaliações para todos os usuários em ambos os conjuntos. Eles serão divididos à proporção de 70% treino e 30% teste, ficando com cerca de 14mi de avaliações no conjunto principal e 6mi de avaliações no conjunto de teste.

In [1]:
# Importando os pacotes necessários
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
#import matplotlib.pyplot as plt
import re

#%matplotlib inline

In [2]:
# Lendo os arquivos
ratings = pd.read_csv('ml-20m/ratings.csv')
movies = pd.read_csv('ml-20m/movies.csv')
#tags = pd.read_csv('ml-20m/tags.csv')

# Mesclando os dados dos filmes com as avaliações
ratings_movies_all = pd.merge(ratings, movies, on='movieId').drop('timestamp', axis=1)

# Separando os índices para os conjuntos de treino e teste
split = StratifiedShuffleSplit(n_splits=1,test_size=0.3)
indices_train, indices_test = next(split.split(np.zeros(len(ratings_movies_all['userId'])), ratings_movies_all['userId']))

# Definindo o conjunto de treino, ou principal
ratings_movies = ratings_movies_all.iloc[indices_train]
ratings_movies

,userId,movieId,rating,title,genres
8595943,96186,3114,3.5,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
8686010,39647,3897,5.0,Almost Famous (2000),Drama
15843113,127138,45720,4.5,"Devil Wears Prada, The (2006)",Comedy|Drama
5098134,125778,594,5.0,Snow White and the Seven Dwarfs (1937),Animation|Children|Drama|Fantasy|Musical
424688,77170,318,4.0,"Shawshank Redemption, The (1994)",Crime|Drama
3607274,63055,1188,4.0,Strictly Ballroom (1992),Comedy|Romance
12996699,8612,1968,3.5,"Breakfast Club, The (1985)",Comedy|Drama
18604761,127342,66371,4.0,Departures (Okuribito) (2008),Drama
15265311,81656,2871,4.5,Deliverance (1972),Adventure|Drama|Thriller
10729807,34853,2997,2.0,Being John Malkovich (1999),Comedy|Drama|Fantasy


In [3]:
# Definindo o conjunto de teste
ratings_movies_test = ratings_movies_all.iloc[indices_test]
ratings_movies_test

,userId,movieId,rating,title,genres
18850884,123265,2618,5.0,"Castle, The (1997)",Comedy
5923207,43657,16,3.0,Casino (1995),Crime|Drama
14236248,17014,1747,3.0,Wag the Dog (1997),Comedy
16427883,17604,3993,4.5,Quills (2000),Drama|Romance
6523843,83694,1617,4.0,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
6638006,90919,2125,2.0,Ever After: A Cinderella Story (1998),Comedy|Drama|Romance
198274,45436,223,3.5,Clerks (1994),Comedy
3279066,19842,457,4.5,"Fugitive, The (1993)",Thriller
13848031,9109,3654,3.0,"Guns of Navarone, The (1961)",Action|Adventure|Drama|War
9327697,33706,30793,2.0,Charlie and the Chocolate Factory (2005),Adventure|Children|Comedy|Fantasy|IMAX


## Passo 2) Como explorar os dados?

Neste ponto a intenção é conhecer melhor os dados, explorando através de questões. Primeiro respondemos questões simples e mais genéricas, como os 10 filmes com mais avaliações 5 estrelas. Depois partimos para questões mais complexas relacionadas a preferência geral.

Deste ponto em diante serão utilizados somente os dados do conjunto de treino para que não haja interferência na avaliação do sistema de recomendação.

In [4]:
# Filtrar os 10 filmes com maior NÚMERO de avaliações 5 estrelas, listando-os pelo título:
top_5star_movies = ratings_movies[ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_movies

Shawshank Redemption, The (1994)             22276
Pulp Fiction (1994)                          19534
Silence of the Lambs, The (1991)             15708
Schindler's List (1993)                      15562
Star Wars: Episode IV - A New Hope (1977)    15455
Forrest Gump (1994)                          14875
Godfather, The (1972)                        14146
Usual Suspects, The (1995)                   13895
Braveheart (1995)                            12932
Matrix, The (1999)                           12929
Name: title, dtype: int64

In [5]:
# Os 20 filmes com maior MÉDIA de estrelas, listando-os pelo título:
top_meanstar = ratings_movies.groupby('title').agg({'rating': [np.size, np.mean]})
top_meanstar.sort_values([('rating', 'mean')], ascending=False).head(20)

rating     
                                                    size mean
title                                                        
Dust in the Wind (Lian lian feng chen) (1986)        1.0  5.0
Yonkers Joe (2008)                                   1.0  5.0
The War at Home (1979)                               1.0  5.0
B-Side (2013)                                        1.0  5.0
The Floating Castle (2012)                           1.0  5.0
Island at War (2004)                                 1.0  5.0
Who Killed Vincent Chin? (1987)                      1.0  5.0
The Green (2011)                                     1.0  5.0
Slasher (2004)                                       1.0  5.0
The House on 56th Street (1933)                      1.0  5.0
Into the Middle of Nowhere (2010)                    1.0  5.0
Snow Queen, The (Lumikuningatar) (1986)              1.0  5.0
Hard Sun (2014)                                      1.0  5.0
Solo Sunny (1980)                                    1.0  5.0
Interrupted Melody (1955)                            1.0  5.0
Victor and the Secret of Crocodile Mansion (2012)    1.0  5.0
Abendland (2011)                                     1.0  5.0
I Belong (Som du ser meg) (2012)                     1.0  5.0
Prom Queen: The Marc Hall Story (2004)               1.0  5.0
De la servitude moderne (2009)                       1.0  5.0

##### Diferença entre usar o número de avaliações 5 estrelas e a média de estrelas por filme:

Ao utilizar a número de avaliações 5 estrelas nós selecionamos os títulos mais populares e bem avaliados, entretanto, podem ficar subamostrados os bons títulos mas com poucas avaliações. Neste ponto supre tal necessidade o uso da média de estrelas por título, mas este último método também oferece a desvantagem de selecionar títulos que não são populares e também títulos que tiveram pouquissimas avaliações, contudo positivas. Por isso, decidiu-se utilizar o número de avaliações 5 estrelas como forma de selecionar filmes bons e populares, sem o risco de oferecer filmes de pouco interesse geral.

### Questões relacionadas aos gêneros

* Quais os filmes com mais avaliações 5 estrelas dentro de cada gênero?

* Quando agregado outro gênero estes mudam radicalmente?


In [6]:
# Um exemplo para filtrar os filmes por gênero e por mais avaliações 5 estrelas, listando-os pelo título:
top_5star_drama = ratings_movies[ratings_movies['genres'].str.contains('Drama')][ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_drama

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Shawshank Redemption, The (1994)    22276
Pulp Fiction (1994)                 19534
Schindler's List (1993)             15562
Forrest Gump (1994)                 14875
Godfather, The (1972)               14146
Braveheart (1995)                   12932
American Beauty (1999)              10998
Fargo (1996)                        10628
Fight Club (1999)                   10209
Godfather: Part II, The (1974)       8160
Name: title, dtype: int64

In [7]:
# Um exemplo para filtrar os filmes por mais de um gênero e por mais avaliações 5 estrelas, listando-os pelo título:
top_5star_drama_romance = ratings_movies[ratings_movies['genres'].str.contains('Drama' and 'Romance')][ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_drama_romance

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Forrest Gump (1994)                                     14875
Princess Bride, The (1987)                               8435
Casablanca (1942)                                        7336
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)     5485
Beauty and the Beast (1991)                              4962
Good Will Hunting (1997)                                 4692
Life Is Beautiful (La Vita è bella) (1997)               4675
Sense and Sensibility (1995)                             4455
Groundhog Day (1993)                                     4421
Shakespeare in Love (1998)                               4370
Name: title, dtype: int64

In [8]:
top_5star_drama_crime = ratings_movies[ratings_movies['genres'].str.contains('Drama' and 'Crime')][ratings_movies['rating'] > 4.5]['title'].value_counts()[0:10]
top_5star_drama_crime

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Shawshank Redemption, The (1994)    22276
Pulp Fiction (1994)                 19534
Silence of the Lambs, The (1991)    15708
Godfather, The (1972)               14146
Usual Suspects, The (1995)          13895
Fargo (1996)                        10628
Fight Club (1999)                   10209
Godfather: Part II, The (1974)       8160
Goodfellas (1990)                    6092
Taxi Driver (1976)                   5572
Name: title, dtype: int64

Através das listas de filmes geradas observamos que a inclusão de um segundo gênero muda sensivelmente os filmes no ranking. Se tal observação se repetir para outras combinações de gêneros, devemos considerar que talvez um gênero não seja o suficiente para geram uma lista de recomendações de filmes para um determinado usuário. 

### Questões relacionadas a preferência por gênero

* Usuários avaliam apenas um gênero, ou mais gêneros? Eles gostam desses gêneros? O quão importante é o gênero na escolha e avaliação do filme?

Para responder a tal pergunta nós seguiremos as seguintes etapas: 

 1. Determinar o número de avaliações por gênero e usuário
 
 2. Determinar a participação de cada gênero no número de avaliações de cada usuário
 
 3. Encontrar quais os gêneros que compõe a maior parte das avaliações do usuário
 
 4. Determinar a avaliação média do usuário para cada gênero identificado
 
 5. Metrificar a importância do gênero na escolha e na avaliação do filme através dos dados anteriores, por exemplo a média de número de gêneros considerados importantes e a médias das avaliações em tais gêneros.



In [9]:
# 1. Determinando o número de avaliações por gênero para cada usuário

# 1a) definindo todos os gêneros que existem:
genre_labels = set()
for s in ratings_movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

# 1b) desmembrando todos os gêneros em colunas separadas
genres_df = pd.DataFrame(dict((genre, ratings_movies['genres'].str.contains(genre, re.IGNORECASE))
                             for genre in genre_labels))
ratings_movies_expand = genres_df.join(ratings_movies)

# 1c) contar quantas vezes os gêneros são verdadeiros por usuário
n_gen_user = ratings_movies_expand.groupby('userId').agg({genre:[np.sum] for genre in genre_labels})

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.


In [10]:
# 2. Determinando a participação percentual de cada gênero no número de avaliações de cada usuário
n_gen_user['total'] = n_gen_user.apply(sum, axis=1)
tax_gen_user = n_gen_user.div(n_gen_user['total'], axis=0).mul(100)

In [11]:
# 3. Encontrando qual(is) gênero(s) que compõe(m) a maior parte das avaliações do usuário

# 3a. Gênero mais importante
tax_pref = tax_gen_user.drop('total', axis=1)
preference = pd.DataFrame()
preference['genre_max1'] = tax_pref.apply(np.argmax, axis=1)
preference['value_max1'] = tax_pref.apply(np.amax, axis=1)

# 3b. Segundo gênero mais importante
i=0
for genre in preference['genre_max1']:
    tax_pref.iloc[i][genre] = -1
    i = i+1

preference['genre_max2'] = tax_pref.apply(np.argmax, axis=1)
preference['value_max2'] = tax_pref.apply(np.amax, axis=1)

preference.head()

C:\Users\mariv\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


,genre_max1,value_max1,genre_max2,value_max2
userId,,,,
1,"(Adventure, sum)",13.888889,"(Fantasy, sum)",13.611111
2,"(Sci-Fi, sum)",17.142857,"(Drama, sum)",14.285714
3,"(Sci-Fi, sum)",20.056497,"(Action, sum)",13.276836
4,"(Thriller, sum)",15.517241,"(Action, sum)",13.793103
5,"(Drama, sum)",15.151515,"(Comedy, sum)",12.878788


In [12]:
# 4. Determinar a avaliação média do usuário para cada gênero identificado

In [13]:
# 5. Metrificar a importância do gênero na escolha e na avaliação do filme

Os pontos 4 e 5 são muito importantes para entender o comportamento dos usuários em geral e assim tomar melhores decisões com relação à recomendação, entretanto acredito que os gêneros mais assitidos pelo usuário já sejam um input importante para um demo de recomendação. A preferência individual pode ser inserida mais tarde de forma a refinar as recomendações baseadas no gênero. 

Por isso, com base na tabela de preferências por usuário, podemos construir um demo do sistema de recomendação baseado nos gêneros identificados.

## Passo 3) Sistema de recomendação

O sistema de recomendação será baseado primeiramente no(s) gênero(s) que o usuário mais avalia, e depois no número de avaliações 5 estrelas dadas pelo universo de usuários.

O sistema seguirá os seguintes passos:

1. Filtrar lista de filmes que possuem o(s) gênero(s) de preferência do usuário e avaliação maior que 4.5

2. Excluir os filmes que o usuário já assitiu

3. Fazer um ranking desta lista de filmes de acordo com o número de avaliações 5 estrelas

4. Sugerir os filmes no topo do ranking (1 ou mais)

In [14]:
# exemplo com um usuário:
user = 3
pref_genre1 = 'Sci-Fi'
pref_genre2 = 'Action'

# 1. Filtrar lista de filmes que possuem o gênero definido e avaliação 5 estrelas
suggestion = ratings_movies[ratings_movies['genres'].str.contains(pref_genre1 and pref_genre2)][ratings_movies['rating'] > 4.5]

# 2. Excluir o que o usuário já assistiu
viewed = ratings_movies[ratings_movies['userId'] == user]['movieId']

for movie in viewed:
    suggestion = suggestion[suggestion['movieId'] != movie]

# 3 e 4. Rankear pelo maior número de avaliações 5 estrelas e sugerir os melhores
n_suggestions = 5
final_suggestion = suggestion['title'].value_counts()[:n_suggestions]
final_suggestion

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


Braveheart (1995)                                        12932
Fight Club (1999)                                        10209
Princess Bride, The (1987)                                8435
Saving Private Ryan (1998)                                7874
Lord of the Rings: The Return of the King, The (2003)     7106
Name: title, dtype: int64

## Passo 3) Avaliação

Para fazer a avaliação será utilizado o conjunto de teste definido no início. Este conjunto contém avaliações de todos os usuários em um percentual de 30% em relação ao total de avaliações. 
A avaliação seguirá os seguintes passos.

1. Determinar se os filmes sugeridos constam no conjunto de teste para cada usuário.

2. Se houver filmes sugeridos no conjunto de testes, somar e fazer a média do número de estrelas dadas a estes filmes pelo usuário.

Estes números obtidos para cada usuário para cada usuário poderá ser manipulado para se tornar um índice único para avaliação do sistema ou avaliado para todos os usuários correlacionando-os com outros dados como o gênero de preferência, ou ainda ao índice de importância do gênero definido na exploração inicial dos dados.


In [15]:
# 1. Determinar se os filmes sugeridos constam no conjunto de teste para cada usuário

# 1a) Listar as avaliações do conjunto de testes do usuário.
test_viewed = ratings_movies_test[ratings_movies_test['userId'] == user] 

# 1b) Verificar o que foi sugerido pelo sistema de recomendação 
evaluate = np.zeros(len(test_viewed), dtype=bool)
i = 0
for movie in test_viewed['title']:
    evaluate[i] = movie in final_suggestion
    i = i+1
test_viewed['saw'] = evaluate
test_viewed.head()

C:\Users\mariv\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,userId,movieId,rating,title,genres,saw
3530772,3,1077,4.0,Sleeper (1973),Comedy|Sci-Fi,False
4458305,3,2657,3.0,"Rocky Horror Picture Show, The (1975)",Comedy|Horror|Musical|Sci-Fi,False
3502437,3,1073,5.0,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical,False
4556760,3,2808,4.0,Universal Soldier (1992),Action|Sci-Fi,False
3492591,3,1060,5.0,Swingers (1996),Comedy|Drama,False


In [16]:
# 2. Se houver filmes sugeridos no conjunto de testes, somar o número de estrelas dadas a estes filmes pelo usuário.
sum_rating_sugg = test_viewed[test_viewed['saw'] == True]['rating'].sum()
mean_rating_sugg = test_viewed[test_viewed['saw'] == True]['rating'].mean()
print('Soma das estrelas dadas aos filmes sugeridos: ', sum_rating_sugg, 'de 25.0')
print('Média de estrelas dadas aos filmes sugeridos: ', mean_rating_sugg)

Soma das estrelas dadas aos filmes sugeridos:  9.0 de 25.0
Média de estrelas dadas aos filmes sugeridos:  4.5
